In [15]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
import time

In [16]:
# 1.获取数据
order_product = pd.read_csv("./29.预测用户对物品喜好案例_数据/order_products__prior.csv")
products = pd.read_csv("./29.预测用户对物品喜好案例_数据/products.csv")
orders = pd.read_csv("./29.预测用户对物品喜好案例_数据/orders.csv")
aisles = pd.read_csv("./29.预测用户对物品喜好案例_数据/aisles.csv")

In [17]:
order_product

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [18]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [19]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [20]:
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [21]:
# 2.数据处理
# 2.1 合并表格
table1 = pd.merge(order_product, products, on=["product_id", "product_id"])
table2 = pd.merge(table1, orders, on=["order_id", "order_id"])
table = pd.merge(table2, aisles, on=["aisle_id", "aisle_id"])

In [22]:
table

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0,eggs
1,26,33120,5,0,Organic Egg Whites,86,16,153404,prior,2,0,16,7.0,eggs
2,120,33120,13,0,Organic Egg Whites,86,16,23750,prior,11,6,8,10.0,eggs
3,327,33120,5,1,Organic Egg Whites,86,16,58707,prior,21,6,9,8.0,eggs
4,390,33120,28,1,Organic Egg Whites,86,16,166654,prior,48,0,12,9.0,eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,3243156,20731,1,0,Straight Sherry,134,5,166400,prior,3,1,13,12.0,specialty wines champagnes
32434485,860862,30582,1,0,Natural Champagne,134,5,104017,prior,13,5,14,18.0,specialty wines champagnes
32434486,1333472,27906,1,0,Imperial Champagne,134,5,62079,prior,10,3,10,10.0,specialty wines champagnes
32434487,2122701,26086,1,1,La Grand Dame Brut Champagne,134,5,77799,prior,2,3,14,3.0,specialty wines champagnes


In [23]:
# 2.2 交叉表合并
table_cross = pd.crosstab(table["user_id"], table["aisle"])

In [24]:
table_cross

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,3,0,0,0,0,2,0,0,0,...,3,1,1,0,0,0,0,2,0,42
3,0,0,0,0,0,0,0,0,0,0,...,4,1,0,0,0,0,0,2,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
206206,0,4,0,0,0,0,4,1,0,0,...,1,0,0,0,0,1,0,1,0,0
206207,0,0,0,0,1,0,0,0,0,0,...,3,4,0,2,1,0,0,11,0,15


In [25]:
# 2.3 数据截取
table_cross_part = table_cross[:1000]

In [42]:
# 3.特征工程 — pca
transfer = PCA(n_components=0.9)
trans_data = transfer.fit_transform(table_cross_part)
# trans_data = table_cross_part

In [43]:
# 4.机器学习（k-means）
st = time.time()
estimator = KMeans(n_clusters=8, random_state=22)
y_predict = estimator.fit_predict(trans_data)

In [44]:
# 5.模型评估
print('轮廓系数法分数：', silhouette_score(trans_data, y_predict))
print('CH系数法分数：', calinski_harabasz_score(trans_data, y_predict))
print(f'训练耗时：{time.time()-st}')

轮廓系数法分数： 0.4640056725989442
CH系数法分数： 204.04966702936417
训练耗时：3.708268165588379


In [41]:
y_predict


array([5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5,
       5, 5, 5, 5, 0, 4, 5, 5, 5, 5, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5,
       5, 4, 5, 4, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 4, 5,
       5, 5, 5, 5, 2, 5, 5, 5, 2, 5, 4, 5, 5, 4, 5, 4, 5, 5, 5, 2, 4, 5,
       4, 0, 4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 2,
       5, 4, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 5, 5, 6, 5, 2, 4, 5, 5, 2, 5, 5, 4, 5, 4, 2, 2, 4,
       4, 5, 5, 5, 5, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 5,
       5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 2, 5,
       5, 4, 5, 4, 5, 2, 5, 4, 5, 5, 0, 6, 5, 5, 5, 1, 5, 4, 5, 5, 4, 4,
       4, 4, 6, 5, 5, 4, 1, 5, 4, 5, 5, 1, 2, 2, 4, 2, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 4, 5, 2,
       5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 2, 4, 2, 2, 5, 5, 4, 5, 5,
       5, 5, 6, 7, 5, 5, 5, 4, 4, 5, 5, 5, 5, 4, 5,